In [100]:
# %pip install pygame

In [101]:
import pygame
import random
import math
import numpy as np

from pygame.locals import (
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)

In [102]:
global ground

# Society attributes
society_attr = {
  "Digitarian": {
    'energy': 100,
    'area': 16,
    'population': 16,
  },
  "Communo": {
    'energy': 999,
    'area': 16,
    'population': 16,
  },
  "Evolutionist": {
    'energy': 100,
    'area': 40,
    'population': 16,
  },
  "Bio": {
    'energy': 100,
    'area': 16,
    'population': 16,
  },
}



In [103]:
# Initialize Pygame
pygame.init()

# Define constants
WINDOW_WIDTH, WINDOW_HEIGHT = 800, 800
GRID_SIZE = 20
# grid_width, grid_height = 40, 40
block_size = 15
agent_size = 5
GRID_WIDTH, GRID_HEIGHT = int(WINDOW_WIDTH // GRID_SIZE), int(WINDOW_HEIGHT // GRID_SIZE)

FPS = 60

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PLAIN_CLR = (33, 33, 33)
FARM_CLR = (89, 69, 69)
CITY_CLR = (18, 75, 135)
LAB_CLR = (42, 87, 68)
NUCLEAR_CLR = (65, 42, 87)

DIGI_CLR = (148, 196, 247)
BIO_CLR = (237, 198, 55)
COMMUNO_CLR = (245, 47, 47)
EVO_CLR = (247, 148, 229)


In [104]:
# Agent class
class Human(pygame.sprite.Sprite):
    def __init__(self, x, y, society, toxic_resistance, resident_idx):
        super().__init__()
        self.image = pygame.Surface((agent_size, agent_size))
        self.rect = self.image.get_rect(center=(x, y))
        self.health = 100
        self.happiness = 50
        self.mutations = 0
        self.toxic_resistance = toxic_resistance
        self.power = 100
        self.society = society
        self.cost = 1    
        match self.society:
            case "Digitarian":
                self.image.fill(DIGI_CLR)
                self.resident_idx = resident_idx
            case "Communo":
                self.image.fill(COMMUNO_CLR)
                self.cost = 0.5    
            case "Evolutionist":
                self.image.fill(EVO_CLR)
                self.positiveMutations = 0
            case "Bio":
                self.image.fill(BIO_CLR)


    def get_ground(self):
        return grid.ground[int((self.rect.centerx - block_size)/GRID_SIZE)][int((self.rect.centery - block_size)/GRID_SIZE)]

    def update(self, agents):
        # neighbours = [(-1,-1), (-1,0), (-1,1), (0,1), (1,1), (1,0), (1,-1), (0,-1)]

        # for i in range(self.rect.x-GRID_SIZE, self.rect.x+GRID_SIZE+1):
        #     for j in range(self.rect.y-GRID_SIZE, self.rect.y+GRID_SIZE+1):
        #         pass

        x = self.rect.x + random.randrange(-GRID_SIZE, GRID_SIZE+1, GRID_SIZE)
        y = self.rect.y + random.randrange(-GRID_SIZE, GRID_SIZE+1, GRID_SIZE)

        print(self.get_ground().agents)
        print(int((self.rect.centerx - block_size)/GRID_SIZE), int((self.rect.centery - block_size)/GRID_SIZE))

        if x >= 10 and x <= WINDOW_WIDTH-10:
            self.get_ground().agents.remove(self)
            self.rect.x = x
            self.get_ground().agents.append(self)
        if y >= 10 and y <= WINDOW_HEIGHT-10:
            self.get_ground().agents.remove(self)
            self.rect.y = y
            self.get_ground().agents.append(self)

    def get_resource(self):
        self.get_ground().generate_energy()
        society_attr[self.society].energy -= self.cost


    # Digitarian
    def detectToxicity(self):
        for i in range(3):
            for j in range(3):
                if (grid.ground[int((self.rect.centerx - block_size)/GRID_SIZE + i)][int((self.rect.centery - block_size)/GRID_SIZE + j)].toxicity > 0):
                    #avoid logic
                    pass
        society_attr[self.society].energy -= self.cost

    
    # Communo
    def share(self):
        agents = self.get_ground().agents
        random = agents[random.randint(agents.length -1)]
        health_share = (random.health + self.health)/2
        happiness_share = (random.happiness + self.happiness)/2
        power_share = (random.happiness + self.happiness)/2
        self.health = health_share
        random.health = health_share
        self.happiness = happiness_share
        random.happiness = happiness_share
        self.power = power_share
        random.power = power_share
        society_attr[self.society].energy -= self.cost

    # Bio
    def fertilizeTile(self):
        self.get_ground().fertility += random.randint(2, 8)
        society_attr[self.society].energy -= self.cost


    # Evolutionist
    def forceMutation(self):
        if(random.random > 0.9):
            self.positiveMutations = self.positiveMutations + 1
        else:
            self.mutations = self.mutations + 1
        society_attr[self.society].energy -= 1


In [105]:
class Grid():
  def __init__(self):
        self.ground = [[None for _ in range(GRID_WIDTH)] for _ in range(GRID_HEIGHT)] 

  def update(self):
    pass

In [106]:
# Ground class
class Ground(pygame.sprite.Sprite):
    def __init__(self, x, y, fertility, toxicity, resources, max_citizens, regeneration ,type):
        super().__init__()
        self.type = type
        self.image = pygame.Surface((block_size, block_size))
        # self.image = pygame.Rect((10,10))
        match type:
            case "plain":
                self.image.fill(PLAIN_CLR)
            case "Farm":
                self.image.fill(FARM_CLR)
            case "City":
                self.image.fill(CITY_CLR)
            case "Lab":
                self.image.fill(LAB_CLR)
            case "Nuclear":
                self.image.fill(NUCLEAR_CLR)
    
        self.rect = self.image.get_rect(center=(x, y))
        self.fertility = fertility
        self.toxicity = toxicity
        self.resources = resources
        self.level = 1
        self.max_citizens = max_citizens
        self.regeneration = regeneration
        self.allow_demutate = False
        self.agents = []
        # Define properties based on type

    def update(self):
        pass
    
    def level_up (self):
        if(self.level == 3):
            return
        self.level = self.level + 1

    #Lab

    def decreaseMutation(self, agent):
        if(self.type == "Lab"):
            if(random.random() > 0.88):
                agent.mutations -= 1
        
    
    # City    
    def generate_energy(self, agent):
        basic = random.randint(5,10)
        energyMultiplier = 1
        decreaseFertility = 1
        match self.type:
            case 'City':
                energyMultiplier += 1
            case 'Lab':
                energyMultiplier += 1
            case 'Nuclear':
                match self.level:
                    case 1:
                        energyMultiplier += 3
                    case 2:
                        energyMultiplier += 6
                    case 3:
                        energyMultiplier += 9
            case 'Farm':
                energyMultiplier += 1
                if(self.level == 3):
                    decreaseFertility = 0
        society_attr[agent.type].energy += basic * energyMultiplier
        if(random.random() > 0.85):
            self.fertility = self.fertility - decreaseFertility
    

    def give_happines(self, agent):
        if (self.level >= 2):
            agent.happines += random.randint(5,10)
        

In [107]:
# Sprite group for agents

grid = Grid()
agents = pygame.sprite.Group()
ground = pygame.sprite.Group()

In [108]:
# Setup Pygame window
window = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Agent Systems Simulation")
clock = pygame.time.Clock()

for i in range(0, GRID_WIDTH):
    for j in range(0, GRID_HEIGHT):
        fertility = random.randint(50,100)
        toxicity = random.randint(0,10)
        resources = random.randint(0,5)
        max_citizens = 1
        regeneration = random.random()
        type = 'plain'
        # g = Ground(i * WINDOW_WIDTH/GRID_WIDTH+block_size, j*WINDOW_HEIGHT/GRID_HEIGHT+block_size, fertility, toxicity, resources, max_citizens, regeneration, type)
        
        g = Ground(i*GRID_SIZE+block_size, j*GRID_SIZE+block_size, fertility, toxicity, resources, max_citizens, regeneration, type)

        # j=x i=y
        grid.ground[j][i] = g

        ground.add(g)


# Create agents
for i in range(16):
    x, y = random.randint(0, GRID_WIDTH//2-1), random.randint(0, GRID_HEIGHT//2-1)
    agent = Human(x*GRID_SIZE+block_size, y*GRID_SIZE+block_size, "Digitarian", random.randint(0, 10), i)
    agents.add(agent)
    grid.ground[x][y].agents.append(agent)
    # print(f'{x} {y}')
    
    x, y = random.randint(GRID_WIDTH//2, GRID_WIDTH-1), random.randint(0, GRID_HEIGHT//2)
    agent = Human(x*GRID_SIZE+block_size, y*GRID_SIZE+block_size, "Bio", random.randint(0, 10), i+16)
    agents.add(agent)
    grid.ground[x][y].agents.append(agent)
    # print(f'{x} {y}')
    
    x, y = random.randint(0, GRID_WIDTH//2-1), random.randint(GRID_HEIGHT//2, GRID_HEIGHT-1)
    agent = Human(x*GRID_SIZE+block_size, y*GRID_SIZE+block_size, "Communo", random.randint(0, 10), i+32)
    agents.add(agent)
    grid.ground[x][y].agents.append(agent)
    # print(f'{x} {y}')
    
    x, y = random.randint(GRID_WIDTH//2-1, GRID_WIDTH-1), random.randint(GRID_HEIGHT//2-1, GRID_HEIGHT-1)
    agent = Human(x*GRID_SIZE+block_size, y*GRID_SIZE+block_size, "Evolutionist", random.randint(0, 10), i+48)
    agents.add(agent)
    grid.ground[x][y].agents.append(agent)
    # print(f'{x} {y}')


# for agent in agents.sprites():
#     print(f'{agent.rect.centerx} {agent.rect.centery}')

# for i in range(GRID_WIDTH):
#     for j in range(GRID_HEIGHT): 
#         if grid.ground[j][i].agents != []:
#             print(f'{grid.ground[j][i].rect.centerx} {grid.ground[j][i].agents[0].rect.centerx}')

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == KEYDOWN:
            if event.key == K_RIGHT:
        # Update
                agents.update(agents)
                ground.update()
                # Draw
                window.fill(BLACK)
                ground.draw(window)
                agents.draw(window)

                pygame.display.flip()
                clock.tick(FPS)

pygame.quit()

[<Human Sprite(in 1 groups)>]
2 5
[<Human Sprite(in 1 groups)>]
20 13
[<Human Sprite(in 1 groups)>]
15 36
[<Human Sprite(in 1 groups)>]
30 27
[<Human Sprite(in 1 groups)>]
16 11
[<Human Sprite(in 1 groups)>]
24 13
[<Human Sprite(in 1 groups)>]
3 26
[<Human Sprite(in 1 groups)>]
30 28
[<Human Sprite(in 1 groups)>]
0 7
[<Human Sprite(in 1 groups)>]
27 8
[<Human Sprite(in 1 groups)>]
0 24
[<Human Sprite(in 1 groups)>]
32 19
[<Human Sprite(in 1 groups)>]
3 1
[<Human Sprite(in 1 groups)>]
34 7
[<Human Sprite(in 1 groups)>]
17 21
[<Human Sprite(in 1 groups)>]
33 32
[<Human Sprite(in 1 groups)>]
13 3
[<Human Sprite(in 1 groups)>]
30 18
[<Human Sprite(in 1 groups)>]
11 20
[<Human Sprite(in 1 groups)>]
27 25
[<Human Sprite(in 1 groups)>]
16 4
[<Human Sprite(in 1 groups)>]
36 16
[<Human Sprite(in 1 groups)>]
13 29
[<Human Sprite(in 1 groups)>]
20 38
[<Human Sprite(in 1 groups)>]
15 9
[<Human Sprite(in 1 groups)>]
34 11
[<Human Sprite(in 1 groups)>]
2 34
[<Human Sprite(in 1 groups)>]
38 29
[<Huma